In [404]:
import os
import json
import pandas as pd
import traceback

from dotenv import load_dotenv

In [405]:
load_dotenv()
API_KEY = os.getenv('OPENROUTER_API_KEY')

In [406]:
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

### Initialization

In [407]:
llm = ChatOpenAI(
    openai_api_key=API_KEY,
    openai_api_base="https://openrouter.ai/api/v1",
    model_name="shisa-ai/shisa-v2-llama3.3-70b:free",
    temperature=0.5,
)

### Response JSON format

In [408]:
RESPONSE_JSON = '''[
  {
    "question": "1. Multiple choice question.",
    "options": {
      "A": "Choice A",
      "B": "Choice B",
      "C": "Choice C",
      "D": "Choice D"
    },
    "answer": "A"
  },
  {
    "question": "2. Multiple choice question.",
    "options": {
      "A": "Choice A",
      "B": "Choice B",
      "C": "Choice C",
      "D": "Choice D"
    },
    "answer": "B"
  },
  {
    "question": "3. Multiple choice question.",
    "options": {
      "A": "Choice A",
      "B": "Choice B",
      "C": "Choice C",
      "D": "Choice D"
    },
    "answer": "C"
  },
  {
    "question": "4. Multiple choice question.",
    "options": {
      "A": "Choice A",
      "B": "Choice B",
      "C": "Choice C",
      "D": "Choice D"
    },
    "answer": "D"
  },
  {
    "question": "5. Multiple choice question.",
    "options": {
      "A": "Choice A",
      "B": "Choice B",
      "C": "Choice C",
      "D": "Choice D"
    },
    "answer": "A"
  }
]'''


### 1st Template (Generation)

In [409]:
TEMPLATE = """
You are an expert question setter with deep knowledge of the subject: {subject}. 
Generate {number} multiple choice questions (MCQs) from the following text. Use a {tone} tone while creating the questions.

TEXT:
{text}

INSTRUCTIONS:
- Each question should have exactly 4 options: A, B, C, and D.
- Only one option should be correct.
- Vary difficulty from easy to hard and cover different parts of the text.
- Ensure relevance and factual accuracy.
- Do not include explanations, code fences, or any extra text.

RESPONSE FORMAT (JSON only—no backticks or markdown):
{response_json}

Return only the raw JSON array, ready for `json.loads()`.
"""


In [410]:
mcq_generation_prompt = PromptTemplate(
    input_variables=['text', 'number', 'subject', 'tone', 'response_json'],
    template=TEMPLATE
)

### 1st Chain

In [411]:
generation_chain = LLMChain(
    llm=llm,
    prompt=mcq_generation_prompt,
    output_key="quiz",
    verbose=True
)

### 2nd Template (review)

In [412]:
TEMPLATE2 = """
You are an expert in English grammar and education, skilled at analyzing and adjusting multiple choice questions (MCQs) for clarity, tone, and student ability.

Subject: {subject}

Below is a JSON array of generated MCQs:
{quiz}

INSTRUCTIONS:
- Check and correct all grammar.
- Ensure each question’s complexity matches the students’ level.
- Maintain relevance to the subject.
- Preserve the specified tone: {tone}.
- If any question is too complex, unclear, or off-tone, revise it while keeping its original intent.

RESPONSE FORMAT (JSON only—no backticks, no markdown, no extra text):
Return the improved quiz as a raw JSON array, identical in structure to the input.
"""


In [413]:
mcq_evaluation_prompt = PromptTemplate(
    input_variables=['quiz', 'subject'],
    template=TEMPLATE2
)

### 2nd Chain

In [414]:
review_chain = LLMChain(
    llm=llm,
    prompt=mcq_evaluation_prompt,
    output_key="reviewed_quiz",
    verbose=True
)

### Combining 1st and 2nd Chains

In [415]:
generate_evaluate_chain = SequentialChain(
    chains=[generation_chain, review_chain],
    input_variables=['text', 'number', 'subject', 'tone', 'response_json'],
    output_variables=['quiz', 'reviewed_quiz'],  # Corrected output_variables spelling and key
    verbose=True  # Moved verbose to SequentialChain itself (if it's supported)
)


### Reading my source data

In [416]:
with open("oop_info.txt", "r", encoding="utf-8") as file:
    source_text = file.read()

In [417]:
source_text

"Object-Oriented Programming (OOP) is a programming paradigm that is based on the concept of objects, which can contain both data in the form of fields (often known as attributes or properties) and methods (functions or procedures) that operate on the data. In OOP, the focus is on organizing code around objects and the interactions between them, rather than on logic and functions. OOP allows programmers to model real-world entities and their relationships, which makes it easier to design, manage, and scale software systems.\n\nA class in OOP is a blueprint for creating objects. It defines the properties (data attributes) and methods (functions) that the objects created from the class will have. The class itself does not hold any data, but rather it provides a template from which individual objects are created. Each object is an instance of a class, and each object can have its own specific values for the attributes defined in the class. This allows objects to represent real-world entit

### Serializing Python dictionary to JSON-formatted-string

In [418]:
json_string = json.dumps(RESPONSE_JSON, separators=(',', ':'))
print(json_string)

"[\n  {\n    \"question\": \"1. Multiple choice question.\",\n    \"options\": {\n      \"A\": \"Choice A\",\n      \"B\": \"Choice B\",\n      \"C\": \"Choice C\",\n      \"D\": \"Choice D\"\n    },\n    \"answer\": \"A\"\n  },\n  {\n    \"question\": \"2. Multiple choice question.\",\n    \"options\": {\n      \"A\": \"Choice A\",\n      \"B\": \"Choice B\",\n      \"C\": \"Choice C\",\n      \"D\": \"Choice D\"\n    },\n    \"answer\": \"B\"\n  },\n  {\n    \"question\": \"3. Multiple choice question.\",\n    \"options\": {\n      \"A\": \"Choice A\",\n      \"B\": \"Choice B\",\n      \"C\": \"Choice C\",\n      \"D\": \"Choice D\"\n    },\n    \"answer\": \"C\"\n  },\n  {\n    \"question\": \"4. Multiple choice question.\",\n    \"options\": {\n      \"A\": \"Choice A\",\n      \"B\": \"Choice B\",\n      \"C\": \"Choice C\",\n      \"D\": \"Choice D\"\n    },\n    \"answer\": \"D\"\n  },\n  {\n    \"question\": \"5. Multiple choice question.\",\n    \"options\": {\n      \"A\": \

### Creating Response and Tracking Token Usage

In [419]:
NUMBER = 5
SUBJECT = 'Object Oriented Programming'
TONE = 'Simple'

In [420]:
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text" : source_text,
            "number" : NUMBER,
            'subject' : SUBJECT,
            'tone' : TONE,
            'response_json' : json_string
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert question setter with deep knowledge of the subject: Object Oriented Programming. 
Generate 5 multiple choice questions (MCQs) from the following text. Use a Simple tone while creating the questions.

TEXT:
Object-Oriented Programming (OOP) is a programming paradigm that is based on the concept of objects, which can contain both data in the form of fields (often known as attributes or properties) and methods (functions or procedures) that operate on the data. In OOP, the focus is on organizing code around objects and the interactions between them, rather than on logic and functions. OOP allows programmers to model real-world entities and their relationships, which makes it easier to design, manage, and scale software systems.

A class in OOP is a blueprint for creating objects. It defines the properties (data attributes) and methods (functions) that the objects create

In [421]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Input Tokens: {cb.prompt_tokens}")      
print(f"Output Tokens: {cb.completion_tokens}")
print(f"Total Costs: {cb.total_cost}")


Total Tokens: 2484
Input Tokens: 1684
Output Tokens: 800
Total Costs: 0.0


### Format the Reviewed Quiz and Store in DataFrame

In [422]:
import json
import pandas as pd

# 1. Get the reviewed_quiz JSON string from your response dict:
reviewed_quiz = response.get('reviewed_quiz')

# 2. Strip any markdown/code-block markers:
if reviewed_quiz.startswith("```json"):
    reviewed_quiz = reviewed_quiz.strip("```json").strip("```").strip()

# 3. Parse it into a Python list of dicts:
parsed_review = json.loads(reviewed_quiz)

In [423]:
parsed_review

[{'question': '1. What is the main focus of Object-Oriented Programming (OOP)?',
  'options': {'A': 'Organizing code based on logic and functions',
   'B': 'Organizing code around objects and their interactions',
   'C': 'Creating functions to manipulate data',
   'D': 'Designing algorithms for problem-solving'},
  'answer': 'B'},
 {'question': '2. What does a class represent in OOP?',
  'options': {'A': 'An instance of an object',
   'B': 'A blueprint for creating objects',
   'C': 'A collection of functions',
   'D': 'A set of data attributes'},
  'answer': 'B'},
 {'question': '3. Which OOP concept combines data and methods into a single unit?',
  'options': {'A': 'Inheritance',
   'B': 'Polymorphism',
   'C': 'Encapsulation',
   'D': 'Abstraction'},
  'answer': 'C'},
 {'question': '4. What does inheritance allow in OOP?',
  'options': {'A': 'Creating multiple objects from one class',
   'B': 'A new class to inherit properties from another class',
   'C': 'Objects to change their cla

In [424]:
# 4. Normalize into a flat DataFrame:
#    This will create columns like 'question', 'options.A', 'options.B', etc.
quiz_data = pd.json_normalize(parsed_review)

In [425]:
quiz_data.head()

,question,answer,options.A,options.B,options.C,options.D
0,1. What is the main focus of Object-Oriented P...,B,Organizing code based on logic and functions,Organizing code around objects and their inter...,Creating functions to manipulate data,Designing algorithms for problem-solving
1,2. What does a class represent in OOP?,B,An instance of an object,A blueprint for creating objects,A collection of functions,A set of data attributes
2,3. Which OOP concept combines data and methods...,C,Inheritance,Polymorphism,Encapsulation,Abstraction
3,4. What does inheritance allow in OOP?,B,Creating multiple objects from one class,A new class to inherit properties from another...,Objects to change their class dynamically,Functions to work with different data types
4,5. Which OOP principle simplifies complex syst...,D,Encapsulation,Inheritance,Polymorphism,Abstraction
